# ner 데이터를 만들어보자..

In [1]:
from dotenv import load_dotenv
import os, openai, re, sys
import pandas as pd
from tqdm import tqdm
sys.path.append(os.path.abspath(".."))
from functions import *
import ast 

/home/hee/anaconda3/envs/korpatE/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1. chatgpt 쿼리

In [17]:
load_dotenv()
api_key = os.getenv("GPT_API_KEY")

In [73]:
data_df = pd.read_csv("../data/target/llm프롬포팅용데이터.csv", encoding='euc-kr', )
data_df = data_df.drop(columns=['seq', 'len_sen'])
data_df = data_df.assign(sen_idx=range(1, len(data_df) + 1)) #TODO: 나중에 데이터 분리할 때부터 idx 추가하는 걸로 변경
data_df.head()

,applicationNumber,sentence,tokens,sen_idx
0,1020190019082,"제 1항에 있어서,상기 시스템은,상기 촬영이미지에 대한 가이드이미지를 생성하는 가이...","[CLS], 제, 1, ##항, ##에, 있, ##어서, ,, 상기, 시스템, ##...",1
1,1020190019082,"제 2항에 있어서,상기 시스템은,상기 촬영이미지와 상기 가이드이미지의 각각의 구도를...","[CLS], 제, 2, ##항, ##에, 있, ##어서, ,, 상기, 시스템, ##...",2
2,1020190043955,수신된 디바이스 정보를 디바이스 등록 정보로 저장하는 디바이스 등록부;새로운 디바이...,"[CLS], 수신, ##된, 디바이스, 정보, ##를, 디바이스, 등록, 정보, #...",3
3,1020190043955,"제 1 항에 있어서,상기 디바이스 접속 결정부는,상기 접속 요청 신호를 수신하는 경...","[CLS], 제, 1, 항, ##에, 있, ##어서, ,, 상기, 디바이스, 접속,...",4
4,1020180000048,"제1항에 있어서,모듈형 컨테이너 부스의 상부에 형성된 전광판은 2단 이상의 복층으로...","[CLS], 제, ##1, ##항, ##에, 있, ##어서, ,, 모듈, ##형, ...",5


In [63]:
# def create_query(sen: str, tokens: str)-> str:
#     base_query = "너는 대한민국에서 가장 유능한 특허 전문가야. 특허 전문가로서 특허 청구항 일부인 아래 문장과 토큰을 보고, 특허 도메인에서 유의미한 값을 갖는 개체명을 찾아줘. 유의미한 개체명이 없는 경우 <None>이라고 답변해줘. 출력은 설명없이 개체명만 ", "로 이어서 출력해줘."
#     result_query = f'{base_query}\n문장:{sen}\n토큰:{tokens}'
#     return result_query

In [64]:
def create_query(sen: str, tokens: str)-> str:
    base_query = "너는 대한민국에서 가장 유능한 특허 전문가야. 특허 전문가로서 특허 청구항 일부인 아래 문장과 토큰을 보고, 특허 도메인에서 유의미한 값을 갖는 개체명을 찾아주고, 토큰 별로 bio태깅을 달아줘. 출력은 설명 없이, {'토큰':'BIO'} 형태의 딕셔너리로 반환해주는데, 코드블록에 주지 말고 바로 대화창에 출력해줘."
    result_query = f'{base_query}\n문장:{sen}\n토큰:{tokens}'
    return result_query

In [65]:
def get_chatGpt_res(query: str)->str:
    openai.api_key = api_key
    model = 'gpt-4o'
    messages = [{
        "role": "system",
        "content": "You are the nation's top patent expert."
    }, {
        "role": "user",
        "content": query
    }]

    response = openai.chat.completions.create(
        messages=messages,
        model=model,
        temperature=0.1  
    )
    answer = response.choices[0].message.content

    return answer

In [ ]:
# patent_bio_dic = {}

# for idx, row in tqdm(data_df.iterrows(), total=len(data_df)):
#     sen, token = row['sentence'], row['tokens']
#     result = get_chatGpt_res(create_query(sen, token)) # 문자열 딕셔너리 
#     patent_bio_dic[row['sen_idx']] = ast.literal_eval(result) # 문자열을 딕셔너리로 변경

# len(patent_bio_dic)

# 기존 데이터 변경

In [2]:
data_path = "../data/target/llm_data_v1.csv"
data_df = pd.read_csv(data_path, encoding='euc-kr')
data_df = data_df.drop(columns=['len_sen', 'seq'])
data_df = data_df.assign(sen_idx=range(1, len(data_df) + 1))
data_df.head()

,applicationNumber,sentence,tokens,entity,sen_idx
0,1020190019082,"제 1항에 있어서,상기 시스템은,상기 촬영이미지에 대한 가이드이미지를 생성하는 가이...","[CLS], 제, 1, ##항, ##에, 있, ##어서, ,, 상기, 시스템, ##...","가이드생성부, 가이드제공부, 가이드제공모듈, 질문입력모듈, 실시간 메이크업 진단 및...",1
1,1020190019082,"제 2항에 있어서,상기 시스템은,상기 촬영이미지와 상기 가이드이미지의 각각의 구도를...","[CLS], 제, 2, ##항, ##에, 있, ##어서, ,, 상기, 시스템, ##...","구도분석부, 유사도분석부, 추가이미지요청부, 가이드분석모듈, 실시간 메이크업 진단 ...",2
2,1020190043955,수신된 디바이스 정보를 디바이스 등록 정보로 저장하는 디바이스 등록부;새로운 디바이...,"[CLS], 수신, ##된, 디바이스, 정보, ##를, 디바이스, 등록, 정보, #...","디바이스 등록부, 디바이스 접속 결정부, 디바이스 인증부, 인증 장치",3
3,1020190043955,"제 1 항에 있어서,상기 디바이스 접속 결정부는,상기 접속 요청 신호를 수신하는 경...","[CLS], 제, 1, 항, ##에, 있, ##어서, ,, 상기, 디바이스, 접속,...","디바이스 접속 결정부, 디바이스 ID 정보, 사용자 ID 정보, 인증키값, 인증 장치",4
4,1020180000048,"제1항에 있어서,모듈형 컨테이너 부스의 상부에 형성된 전광판은 2단 이상의 복층으로...","[CLS], 제, ##1, ##항, ##에, 있, ##어서, ,, 모듈, ##형, ...","모듈형 컨테이너 부스, 전광판, 날씨정보, 광고 및 홍보정보, 안전사고정보, 내진모...",5


In [42]:
# tokens, bio 리스트 피기
result_df = data_df.copy()

def str_to_list(data: str)->list:
    result_list = [token.strip() for token in data.split(",")]
    return result_list

result_df['tokens'] = result_df['tokens'].apply(lambda data: str_to_list(data)) # 문자열을 리스트로 변경
result_df['entity'] = result_df['entity'].apply(lambda data: str_to_list(data))

In [48]:
model_path = "../KIPI-KorPatELECTRA/KorPatELECTRA/PT"
tokenizer = ElectraTokenizer.from_pretrained(model_path)
def tokenizing(sen: str)-> str:
    tokens = tokenizer.tokenize(sen)
    return tokens

def tag_entity(tokens, entities):
    bio_tags = ["O"] * len(tokens) 

    for entity in entities:
        entity_tokens = tokenizing(entity)  
        entity_len = len(entity_tokens)

        for i in range(len(tokens) - entity_len + 1):
            if tokens[i:i+entity_len] == entity_tokens:  
                bio_tags[i] = "B"  
                for j in range(1, entity_len):
                    bio_tags[i + j] = "I"  
    
    return bio_tags

def bio_tagging(row):
    return tag_entity(row['tokens'], row['entity'])
    
result_df['bio'] = None
result_df['bio'] = result_df.apply(bio_tagging, axis=1)
result_df

,applicationNumber,sentence,tokens,entity,sen_idx,bio
0,1020190019082,"제 1항에 있어서,상기 시스템은,상기 촬영이미지에 대한 가이드이미지를 생성하는 가이...","[[CLS], 제, 1, ##항, ##에, 있, ##어서, , , 상기, 시스템, ...","[가이드생성부, 가이드제공부, 가이드제공모듈, 질문입력모듈, 실시간 메이크업 진단 ...",1,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,1020190019082,"제 2항에 있어서,상기 시스템은,상기 촬영이미지와 상기 가이드이미지의 각각의 구도를...","[[CLS], 제, 2, ##항, ##에, 있, ##어서, , , 상기, 시스템, ...","[구도분석부, 유사도분석부, 추가이미지요청부, 가이드분석모듈, 실시간 메이크업 진단...",2,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,1020190043955,수신된 디바이스 정보를 디바이스 등록 정보로 저장하는 디바이스 등록부;새로운 디바이...,"[[CLS], 수신, ##된, 디바이스, 정보, ##를, 디바이스, 등록, 정보, ...","[디바이스 등록부, 디바이스 접속 결정부, 디바이스 인증부, 인증 장치]",3,"[O, O, O, O, O, O, O, O, O, O, O, O, O, B, I, ..."
3,1020190043955,"제 1 항에 있어서,상기 디바이스 접속 결정부는,상기 접속 요청 신호를 수신하는 경...","[[CLS], 제, 1, 항, ##에, 있, ##어서, , , 상기, 디바이스, 접...","[디바이스 접속 결정부, 디바이스 ID 정보, 사용자 ID 정보, 인증키값, 인증 장치]",4,"[O, O, O, O, O, O, O, O, O, O, B, I, I, O, O, ..."
4,1020180000048,"제1항에 있어서,모듈형 컨테이너 부스의 상부에 형성된 전광판은 2단 이상의 복층으로...","[[CLS], 제, ##1, ##항, ##에, 있, ##어서, , , 모듈, ##형...","[모듈형 컨테이너 부스, 전광판, 날씨정보, 광고 및 홍보정보, 안전사고정보, 내진...",5,"[O, O, O, O, O, O, O, O, O, B, I, I, I, O, O, ..."
...,...,...,...,...,...,...
178,1020040101224,"제2항에 있어서, 상기 멀티...","[[CLS], 제, ##2, ##항, ##에, 있, ##어서, , , 상기, 멀티,...",[멀티 콘솔 컨트롤러 OR 게이트 먹스 MUX 멀티 사용 하드웨어 드라이버 Mult...,179,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
179,1020040101224,"제2항에 있어서, 상기 콘솔...","[[CLS], 제, ##2, ##항, ##에, 있, ##어서, , , 상기, 콘솔,...",[콘솔서버모듈 매직미러 콘솔서버모듈 세톱박스 콘솔서버모듈 홈오토메이션 콘솔서버모듈 ...,180,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
180,1020040101224,"제5항에 있어서, 상기 콘솔...","[[CLS], 제, ##5, ##항, ##에, 있, ##어서, , , 상기, 콘솔,...",[콘솔서버모듈 연결단자 무선 액세스포인트 매직미러 콘솔 세톱박스 콘솔 홈오토메이션 ...,181,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
181,1020040101224,"제5항에 있어서, 상기 서비...","[[CLS], 제, ##5, ##항, ##에, 있, ##어서, , , 상기, 서비스...","[서비스 콘솔, 콘솔서버모듈, 인터넷 애견관리장치, 날씨 정보기, 욕조 수위 조절기...",182,"[O, O, O, O, O, O, O, O, O, O, B, I, O, O, O, ..."


In [49]:
expanded_df = result_df.explode(["tokens", "bio"]).reset_index(drop=True) # 리스트를 행으로 확장
expanded_df

,applicationNumber,sentence,tokens,entity,sen_idx,bio
0,1020190019082,"제 1항에 있어서,상기 시스템은,상기 촬영이미지에 대한 가이드이미지를 생성하는 가이...",[CLS],"[가이드생성부, 가이드제공부, 가이드제공모듈, 질문입력모듈, 실시간 메이크업 진단 ...",1,O
1,1020190019082,"제 1항에 있어서,상기 시스템은,상기 촬영이미지에 대한 가이드이미지를 생성하는 가이...",제,"[가이드생성부, 가이드제공부, 가이드제공모듈, 질문입력모듈, 실시간 메이크업 진단 ...",1,O
2,1020190019082,"제 1항에 있어서,상기 시스템은,상기 촬영이미지에 대한 가이드이미지를 생성하는 가이...",1,"[가이드생성부, 가이드제공부, 가이드제공모듈, 질문입력모듈, 실시간 메이크업 진단 ...",1,O
3,1020190019082,"제 1항에 있어서,상기 시스템은,상기 촬영이미지에 대한 가이드이미지를 생성하는 가이...",##항,"[가이드생성부, 가이드제공부, 가이드제공모듈, 질문입력모듈, 실시간 메이크업 진단 ...",1,O
4,1020190019082,"제 1항에 있어서,상기 시스템은,상기 촬영이미지에 대한 가이드이미지를 생성하는 가이...",##에,"[가이드생성부, 가이드제공부, 가이드제공모듈, 질문입력모듈, 실시간 메이크업 진단 ...",1,O
...,...,...,...,...,...,...
21123,1020040101224,"제5항에 있어서, 상기 서비...",콘솔,"[서비스 콘솔, 콘솔서버모듈, 인터넷 애견관리장치, 날씨 정보기, 욕조 수위 조절기...",183,I
21124,1020040101224,"제5항에 있어서, 상기 서비...",##서버,"[서비스 콘솔, 콘솔서버모듈, 인터넷 애견관리장치, 날씨 정보기, 욕조 수위 조절기...",183,I
21125,1020040101224,"제5항에 있어서, 상기 서비...",시스템,"[서비스 콘솔, 콘솔서버모듈, 인터넷 애견관리장치, 날씨 정보기, 욕조 수위 조절기...",183,I
21126,1020040101224,"제5항에 있어서, 상기 서비...",.,"[서비스 콘솔, 콘솔서버모듈, 인터넷 애견관리장치, 날씨 정보기, 욕조 수위 조절기...",183,O


In [50]:
expanded_df.to_csv("../data/output/new_data2.csv", encoding='euc-kr')

# 폐기

## 1. bio  사전으로, bio 태깅 만들기
- 딕셔너리로 하니까, 아닌 부분도 b, i 태깅이 되버림. 
- 가이드:B => '가이드'생성부, '가이드'제공부.. but, '가이드'이미지까지 되버림

In [29]:
# bio 사전 만들기 
token_bio_dic = {}

model_path = "../KIPI-KorPatELECTRA/KorPatELECTRA/PT"
tokenizer = ElectraTokenizer.from_pretrained(model_path)
def tokenizing(sen: str)-> str:
    tokens = tokenizer.tokenize(sen)
    return tokens
    
sen_group = expanded_df.groupby('sen_idx')
delete_word = [',']

for idx, group in sen_group:
    entity = group['entity'].iloc[0]
    entity_tokenizing = tokenizing(entity)
    entity_tokenizing = [entity for entity in entity_tokenizing if entity not in delete_word]
    entity_list = [entity.strip() for entity in entity.split(",")]
    
    bio_dic = {}
    for e in entity_list:  # 가이드생성부
        b_flag = True
        token_count = 0
        for token in entity_tokenizing: # 가이드, ##생성부
            new_token = token.replace("#", "")
            if b_flag:
                target_e = e[:len(new_token)]; before_length = len(new_token)
                bio_dic[token] = "B"
                b_flag = False; token_count+=1
            else:
                target_e = e[before_length:before_length+len(new_token)]
                before_length+=len(new_token)
                bio_dic[token] = "I"
                token_count+=1
                if before_length==len(e): 
                    entity_tokenizing = entity_tokenizing[token_count:]
    token_bio_dic[idx] = bio_dic

len(token_bio_dic), token_bio_dic[1]

(183,
 {'가이드': 'B',
  '##생성부': 'I',
  '##제공부': 'I',
  '##제공': 'I',
  '##모듈': 'I',
  '질문': 'B',
  '##입력': 'I',
  '실시간': 'B',
  '메이크업': 'I',
  '진단': 'I',
  '및': 'I',
  '질의': 'I',
  '##응답': 'I',
  '서비스': 'I',
  '제공': 'I',
  '시스템': 'I'})

In [31]:
expanded_df['bio'] = None
for idx, row in tqdm(expanded_df.iterrows(), total=len(expanded_df)):
    bio_dic = token_bio_dic[row['sen_idx']]

    expanded_df.loc[idx, 'bio'] = bio_dic.get(row['tokens'], "O")
    
expanded_df

100%|██████████| 20349/20349 [00:03<00:00, 5488.44it/s]


,applicationNumber,sentence,tokens,entity,sen_idx,bio
0,1020190019082,"제 1항에 있어서,상기 시스템은,상기 촬영이미지에 대한 가이드이미지를 생성하는 가이...",[CLS],"가이드생성부, 가이드제공부, 가이드제공모듈, 질문입력모듈, 실시간 메이크업 진단 및...",1,O
1,1020190019082,"제 1항에 있어서,상기 시스템은,상기 촬영이미지에 대한 가이드이미지를 생성하는 가이...",제,"가이드생성부, 가이드제공부, 가이드제공모듈, 질문입력모듈, 실시간 메이크업 진단 및...",1,O
2,1020190019082,"제 1항에 있어서,상기 시스템은,상기 촬영이미지에 대한 가이드이미지를 생성하는 가이...",1,"가이드생성부, 가이드제공부, 가이드제공모듈, 질문입력모듈, 실시간 메이크업 진단 및...",1,O
3,1020190019082,"제 1항에 있어서,상기 시스템은,상기 촬영이미지에 대한 가이드이미지를 생성하는 가이...",##항,"가이드생성부, 가이드제공부, 가이드제공모듈, 질문입력모듈, 실시간 메이크업 진단 및...",1,O
4,1020190019082,"제 1항에 있어서,상기 시스템은,상기 촬영이미지에 대한 가이드이미지를 생성하는 가이...",##에,"가이드생성부, 가이드제공부, 가이드제공모듈, 질문입력모듈, 실시간 메이크업 진단 및...",1,O
...,...,...,...,...,...,...
20344,1020040101224,"제5항에 있어서, 상기 서비...",콘솔,"서비스 콘솔, 콘솔서버모듈, 인터넷 애견관리장치, 날씨 정보기, 욕조 수위 조절기,...",183,I
20345,1020040101224,"제5항에 있어서, 상기 서비...",##서버,"서비스 콘솔, 콘솔서버모듈, 인터넷 애견관리장치, 날씨 정보기, 욕조 수위 조절기,...",183,I
20346,1020040101224,"제5항에 있어서, 상기 서비...",시스템,"서비스 콘솔, 콘솔서버모듈, 인터넷 애견관리장치, 날씨 정보기, 욕조 수위 조절기,...",183,I
20347,1020040101224,"제5항에 있어서, 상기 서비...",.,"서비스 콘솔, 콘솔서버모듈, 인터넷 애견관리장치, 날씨 정보기, 욕조 수위 조절기,...",183,O


In [33]:
expanded_df.to_csv("../data/output/new_data1.csv", encoding='euc-kr')